In [1]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy
import pandas as pd
#from xgboost import XGBClassifier

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [3]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [4]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams(aggregated_data)
# Run script

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

exported_pipeline = LogisticRegression()

exported_pipeline.fit(np.array(X), np.array(Y))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
# perform simluation of 2017 NCAA tourney
print("Simulating 2017 NCAA tourney")
submission_data = []
tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
tourney_2017_teams = tourney_seeds[tourney_seeds['Season'] == 2017]['Team']

stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
print("Predicting matchups.")
tourney_2017_teams.sort_values()

for team_1 in tourney_2017_teams:
    for team_2 in tourney_2017_teams:
        if team_1 != team_2:
            prediction = mm_predictor.predict_winner(
                team_1, team_2, exported_pipeline, 2017, stat_fields)
            label = '2017_' + str(team_1) + '_' + str(team_2)
            submission_data.append([label, prediction[0][0]])

Simulating 2017 NCAA tourney
Predicting matchups.


In [18]:
teamNames = pd.read_csv('./ncaa-data/Teams.csv')

team_id_map = {}
for index, row in teamNames.iterrows():
    team_id_map[row['TeamID']] = row['TeamName']

teamTeamWinChanceMap = {}
for line in submission_data:
    title = line[0].split('_')
    team1 = team_id_map[int(title[1])]
    team2 = team_id_map[int(title[2])]
    winRate = str(round(line[1]* 100,2))
    if team1 not in teamTeamWinChanceMap:
        teamTeamWinChanceMap[team1] = {}
    if float(winRate) >= 50:
        teamTeamWinChanceMap[team1][team2] = float(winRate)
        print(team1 + ' beats ' + team2 + ': ' + str(winRate))

print(teamTeamWinChanceMap)

Villanova beats Duke: 73.07
Villanova beats Baylor: 81.92
Villanova beats Florida: 81.07
Villanova beats Virginia: 83.03
Villanova beats SMU: 65.63
Villanova beats South Carolina: 87.81
Villanova beats Wisconsin: 82.12
Villanova beats Virginia Tech: 83.9
Villanova beats Marquette: 87.43
Villanova beats Providence: 85.45
Villanova beats USC: 85.54
Villanova beats UNC Wilmington: 77.13
Villanova beats ETSU: 81.5
Villanova beats New Mexico St: 82.23
Villanova beats Troy: 87.58
Villanova beats Mt St Mary's: 91.04
Villanova beats New Orleans: 90.51
Villanova beats Gonzaga: 64.35
Villanova beats Arizona: 65.47
Villanova beats Florida St: 80.41
Villanova beats West Virginia: 79.17
Villanova beats Notre Dame: 79.63
Villanova beats Maryland: 83.9
Villanova beats St Mary's CA: 74.02
Villanova beats Northwestern: 85.45
Villanova beats Vanderbilt: 85.88
Villanova beats VA Commonwealth: 79.4
Villanova beats Xavier: 88.47
Villanova beats Princeton: 79.17
Villanova beats Bucknell: 82.83
Villanova bea

SMU beats Villanova: 63.33
SMU beats Duke: 72.04
SMU beats Baylor: 81.14
SMU beats Florida: 80.27
SMU beats Virginia: 82.3
SMU beats South Carolina: 87.25
SMU beats Wisconsin: 81.36
SMU beats Virginia Tech: 83.2
SMU beats Marquette: 86.85
SMU beats Providence: 84.8
SMU beats USC: 84.89
SMU beats UNC Wilmington: 76.21
SMU beats ETSU: 80.71
SMU beats New Mexico St: 81.46
SMU beats Troy: 87.01
SMU beats Mt St Mary's: 90.61
SMU beats New Orleans: 90.06
SMU beats Gonzaga: 63.17
SMU beats Arizona: 64.3
SMU beats Florida St: 79.59
SMU beats West Virginia: 78.31
SMU beats Notre Dame: 78.78
SMU beats Maryland: 83.2
SMU beats St Mary's CA: 73.02
SMU beats Northwestern: 84.8
SMU beats Vanderbilt: 85.25
SMU beats VA Commonwealth: 78.55
SMU beats Xavier: 87.93
SMU beats Princeton: 78.31
SMU beats Bucknell: 82.09
SMU beats FL Gulf Coast: 83.0
SMU beats North Dakota: 87.09
SMU beats S Dakota St: 90.18
SMU beats Kansas: 69.13
SMU beats Louisville: 78.43
SMU beats Oregon: 70.46
SMU beats Purdue: 77.22


North Dakota beats Northwestern: 58.58
North Dakota beats Vanderbilt: 59.43
North Dakota beats Xavier: 64.88
North Dakota beats Bucknell: 53.75
North Dakota beats FL Gulf Coast: 55.31
North Dakota beats S Dakota St: 69.96
North Dakota beats Creighton: 54.97
North Dakota beats Miami FL: 56.18
North Dakota beats Michigan St: 65.04
North Dakota beats Oklahoma St: 60.78
North Dakota beats Iona: 63.26
North Dakota beats Jacksonville St: 71.13
North Dakota beats NC Central: 65.83
North Dakota beats UC Davis: 67.85
North Dakota beats Butler: 53.4
North Dakota beats Minnesota: 52.17
North Dakota beats Dayton: 50.59
North Dakota beats Arkansas: 50.94
North Dakota beats Seton Hall: 54.1
North Dakota beats Kansas St: 66.15
North Dakota beats Wake Forest: 62.28
North Dakota beats Winthrop: 52.0
North Dakota beats Kent: 60.61
North Dakota beats N Kentucky: 57.9
North Dakota beats TX Southern: 63.26
S Dakota St beats South Carolina: 55.66
S Dakota St beats Marquette: 54.8
S Dakota St beats Providenc

Kansas St beats FL Gulf Coast: 51.85
Kansas St beats North Dakota: 59.8
Kansas St beats S Dakota St: 66.95
Kansas St beats Creighton: 51.5
Kansas St beats Miami FL: 52.72
Kansas St beats Michigan St: 61.8
Kansas St beats Oklahoma St: 57.41
Kansas St beats Iona: 59.96
Kansas St beats Jacksonville St: 68.18
Kansas St beats NC Central: 62.63
Kansas St beats UC Davis: 64.74
Kansas St beats Seton Hall: 50.62
Kansas St beats Wake Forest: 58.95
Kansas St beats Kent: 57.24
Kansas St beats N Kentucky: 54.47
Kansas St beats TX Southern: 59.96
Wake Forest beats Baylor: 53.09
Wake Forest beats Florida: 51.69
Wake Forest beats Virginia: 55.01
Wake Forest beats South Carolina: 64.27
Wake Forest beats Wisconsin: 53.44
Wake Forest beats Virginia Tech: 56.56
Wake Forest beats Marquette: 63.46
Wake Forest beats Providence: 59.47
Wake Forest beats USC: 59.64
Wake Forest beats ETSU: 52.39
Wake Forest beats New Mexico St: 53.61
Wake Forest beats Troy: 63.79
Wake Forest beats Mt St Mary's: 71.73
Wake Forest

In [8]:
print("Writing %d results." % len(submission_data))
with open('./ncaa-data/submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'pred'])
    writer.writerows(submission_data)

Writing 4556 results.


In [9]:
tourney_data2017 = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data2017 = tourney_data2017[tourney_data2017.Season == 2017]

winners = []

tourney_data_grouped = tourney_data2017[4:].groupby('WTeamID').size().reset_index(name='NumWins')
for index, row in tourney_data_grouped.iterrows():
    teamName = team_id_map[row['WTeamID']]
    wins = row['NumWins']
    if len(winners) == 0:
        winners.append([])
    winners[0].append(teamName)
    if wins > 1:
        if len(winners) == 1:
            winners.append([])
        winners[1].append(teamName)
    if wins > 2:
        if len(winners) == 2:
            winners.append([])
        winners[2].append(teamName)
    if wins > 3:
        if len(winners) == 3:
            winners.append([])
        winners[3].append(teamName)
    if wins > 4:
        if len(winners) == 4:
            winners.append([])
        winners[4].append(teamName)
    if wins > 5:
        if len(winners) == 5:
            winners.append([])
        winners[5].append(teamName)

In [10]:
tourney_data2017

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
914,2017,134,1243,95,1448,88,N,0,31,47,...,27,29,32,7,14,14,11,8,1,25
915,2017,134,1291,67,1309,66,N,0,25,47,...,9,7,12,10,18,12,14,10,2,18
916,2017,135,1413,67,1300,63,N,0,22,48,...,26,10,13,9,21,14,9,10,0,15
917,2017,135,1425,75,1344,71,N,0,24,57,...,20,12,17,7,20,17,9,3,2,21
918,2017,136,1112,100,1315,82,N,0,36,62,...,22,8,16,9,22,18,6,6,2,23
919,2017,136,1139,76,1457,64,N,0,24,49,...,25,5,10,3,24,12,9,6,1,22
920,2017,136,1196,80,1190,65,N,0,26,54,...,24,6,10,13,19,12,17,9,0,19
921,2017,136,1199,86,1195,80,N,0,30,54,...,28,14,20,7,19,18,11,6,1,23
922,2017,136,1211,66,1355,46,N,0,25,63,...,20,5,11,9,29,8,13,3,4,17
923,2017,136,1235,84,1305,73,N,0,30,54,...,26,13,16,10,18,13,7,8,4,14


In [11]:
# First four rows is 'first four' and not in actual bracket
firstFour = tourney_data2017[:4]
mainTourney = tourney_data2017[4:]

tourney = [-1] * max(tourney_seeds['Team'])

# Look at first four teams
for index, row in firstFour.iterrows():
    index1 = int(row['WTeamID'])
    index2 = int(row['LTeamID'])
    team1Name = team_id_map[index1]
    team2Name = team_id_map[index2]
    
    if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]):
        tourney[index2] = index1
        tourney[index1] = tourney[index1] - 1
    else:
        tourney[index1] = index2
        tourney[index2] = tourney[index2] - 1

# MAIN TOURNAMENT 

tourney = [-1] * max(tourney_seeds['Team'])
score = 0
for index, row in mainTourney.iterrows():
    index1 = int(row['WTeamID'])
    index2 = int(row['LTeamID'])
    while tourney[index1] > 0:
        index1 = tourney[index1]
    while tourney[index2] > 0:
        index2 = tourney[index2]
    team1Name = team_id_map[index1]
    team2Name = team_id_map[index2]
    
    if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]): # team1 would win
        tourney[index2] = index1
        tourney[index1] = tourney[index1] - 1
        if team1Name in winners[abs(tourney[index1]) - 2]:
            score += (abs(tourney[index1]) - 1) * 10
        print(team1Name + ' vs ' + team2Name + ', team 1 wins')
    else: #team2 would win
        tourney[index1] = index2
        tourney[index2] = tourney[index2] - 1
        if team2Name in winners[abs(tourney[index2]) - 2]:
            score += (abs(tourney[index2]) - 1) * 10
        print(team1Name + ' vs ' + team2Name + ', team 2 wins')
print(score)

Arizona vs North Dakota, team 1 wins
Butler vs Winthrop, team 1 wins
Florida vs ETSU, team 1 wins
Florida St vs FL Gulf Coast, team 1 wins
Gonzaga vs S Dakota St, team 1 wins
Iowa St vs Nevada, team 1 wins
MTSU vs Minnesota, team 1 wins
Northwestern vs Vanderbilt, team 1 wins
Notre Dame vs Princeton, team 1 wins
Purdue vs Vermont, team 1 wins
St Mary's CA vs VA Commonwealth, team 1 wins
Villanova vs Mt St Mary's, team 1 wins
Virginia vs UNC Wilmington, team 1 wins
West Virginia vs Bucknell, team 1 wins
Wisconsin vs Virginia Tech, team 1 wins
Xavier vs Maryland, team 1 wins
Arkansas vs Seton Hall, team 1 wins
Baylor vs New Mexico St, team 1 wins
Cincinnati vs Kansas St, team 1 wins
Duke vs Troy, team 1 wins
Kansas vs UC Davis, team 1 wins
Kentucky vs N Kentucky, team 1 wins
Louisville vs Jacksonville St, team 1 wins
Michigan vs Oklahoma St, team 1 wins
Michigan St vs Miami FL, team 1 wins
North Carolina vs TX Southern, team 1 wins
Oregon vs Iona, team 1 wins
Rhode Island vs Creighton, t

In [12]:
teamTeamWinChanceMap

{'Arizona': {'Arkansas': 80.49,
  'Baylor': 81.26,
  'Bucknell': 82.2,
  'Butler': 81.99,
  'Cincinnati': 73.03,
  'Creighton': 82.91,
  'Dayton': 80.27,
  'Duke': 72.19,
  'ETSU': 80.82,
  'FL Gulf Coast': 83.1,
  'Florida': 80.38,
  'Florida St': 79.71,
  'Gonzaga': 63.34,
  'Iona': 87.25,
  'Iowa St': 78.2,
  'Jacksonville St': 90.73,
  'Kansas': 69.29,
  'Kansas St': 88.59,
  'Kent': 85.95,
  'Kentucky': 66.85,
  'Louisville': 78.55,
  'MTSU': 73.58,
  'Marquette': 86.93,
  'Maryland': 83.3,
  'Miami FL': 83.59,
  'Michigan': 78.79,
  'Michigan St': 88.08,
  'Minnesota': 81.26,
  "Mt St Mary's": 90.67,
  'N Kentucky': 84.53,
  'NC Central': 88.45,
  'Nevada': 75.05,
  'New Mexico St': 81.57,
  'New Orleans': 90.12,
  'North Carolina': 74.26,
  'North Dakota': 87.17,
  'Northwestern': 84.9,
  'Notre Dame': 78.9,
  'Oklahoma St': 86.03,
  'Oregon': 70.62,
  'Princeton': 78.43,
  'Providence': 84.9,
  'Purdue': 77.35,
  'Rhode Island': 77.95,
  'S Dakota St': 90.25,
  'SMU': 64.63,
  